## 几种问题的遍历方法

### 调库 与 位运算实现

In [109]:
import numpy as np
import time
# 对 i 从 0 到 2^m-1 遍历 第 n 位为 1 代表选取该位
def find(n, i): return (i & (1 << n)) > 0

### 背包问题

#### 代码

In [110]:
# N: 总个数
# S: 总承重
# s: 物品重量向量
# c: 物品价值向量
def traverse_bag(N, S, s, c):
    best = 0
    m = 0
    for i in range(2 ** N):
        s_now = 0
        m_now = 0
        for k in range(N):
            if find(k, i):
                s_now += s[k]
                m_now += c[k]
        if s_now < S and m_now > m:
            m = m_now
            best = i
    print("best cost: " + str(m))
    print("best choice: " + format(best, '0' + str(N) + 'b')[::-1])
    return m, best


#### 测试

In [111]:
def test(N):
    S = N * 10
    print("N = " + str(N) + ", S = " + str(S))
    s = np.random.randint(5, 20, size = N)
    print(s)
    c = np.random.randint(1, 100, size = N)
    print(c)
    start = time.time()
    traverse_bag(N, S, s, c)
    end = time.time()
    print(f"time: {end - start}s")
    print("")

test(10)
test(15)
test(20)
test(25)

N = 10, S = 100
[ 9 18  9 11 19 17 16 18  6  7]
[34 52 61 72  2 38  2 24 35 74]
best cost: 390
best choice: 1111010111
time: 0.0019998550415039062s

N = 15, S = 150
[ 9 16  5  5 17  7 13 12 16 15 12 19 13 15 16]
[93 54 15 68 56 67 76  1 28 98  1 23 21 85 97]
best cost: 758
best choice: 111111101100111
time: 0.07551312446594238s

N = 20, S = 200
[16 13 16  6 17 14 13  6  6 17 12 14  9  9 10  6  5 19 15 19]
[77 80 59 80 72 47 29 61  2 13 94 57 24  7 84 43  5 70 97  8]
best cost: 981
best choice: 11111111001111110110
time: 3.061936140060425s

N = 25, S = 250
[11  8  6  7 11  8 19 16 19 10 13  9  8  9 13 11 10 15  8 10  7 12 12 17
  6]
[79 70 23 22 92 92 22 70 87 81 64 55 10 92 21 25 63 19 28 92 97 21 36 58
 84]
best cost: 1371
best choice: 1111110111110111111111111
time: 121.68455338478088s



### 最大割问题

#### 代码

In [112]:
# N: 顶点个数 编号为 0 到 N-1 默认源点为 0，汇点为 N-1
# E: 边集合 为一些满足 i < j 二元向量 (i, j) 构成
def traverse_maxcut(N, E):
    best = 0
    m = 0
    for i in range(2 ** (N-2)):
        m_now = 0
        for e in E:
            if e[0] == 0 or find(e[0] - 1, i):
                if e[1] == N - 1 or not find(e[1] - 1, i):
                    m_now += 1
            elif find(e[1] - 1, i):
                m_now += 1
        if m_now > m:
            m = m_now
            best = i
    print("max edges: " + str(m))
    print("best choice: 1" + format(best, '0' + str(N-2) + 'b')[::-1] + "0")
    return m, best

#### 测试

In [113]:
def test2(N, M):
    E = []
    rnds = np.random.choice(range(N * (N - 1) // 2), M, replace = False)
    count = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            if count in rnds:
                E.append([i, j])
            count += 1
    print("N = " + str(N) +  ", M = " + str(M))
    # print(E)
    start = time.time()
    traverse_maxcut(N, E)
    end = time.time()
    print(f"time: {end - start}s")
    print("")

test2(10, 10)
test2(10, 45)
test2(15, 50)
test2(15, 100)
test2(20, 10)
test2(20, 100)
test2(25, 30)
test2(25, 300)

N = 10, M = 10
max edges: 9
best choice: 1101100100
time: 0.0010099411010742188s

N = 10, M = 45
max edges: 25
best choice: 1111100000
time: 0.0019969940185546875s

N = 15, M = 50
max edges: 35
best choice: 110011010011100
time: 0.07805824279785156s

N = 15, M = 100
max edges: 56
best choice: 110111010100000
time: 0.13410186767578125s

N = 20, M = 10
max edges: 10
best choice: 10101001100100110000
time: 0.4873371124267578s

N = 20, M = 100
max edges: 65
best choice: 11011010001101100000
time: 4.537440776824951s

N = 25, M = 30
max edges: 28
best choice: 1101000000100011010100100
time: 44.50195860862732s

N = 25, M = 300
max edges: 156
best choice: 1111111111110000000000000
time: 442.4826683998108s



### Cheeger问题

#### 代码

In [116]:
# N: 顶点个数 编号为 0 到 N-1 默认源点为 0，汇点为 N-1
# E: 边集合 为一些满足 i < j 二元向量 (i, j) 构成
def traverse_cheeger(N, E):
    D = np.zeros(N, dtype=np.int32)
    for e in E:
        D[e[0]] += 1
        D[e[1]] += 1
    best = 0
    m = 1
    for i in range(2 ** (N-2)):
        m_now = 0
        ds = D[0]
        for e in E:
            if e[0] == 0 or find(e[0] - 1, i):
                if e[1] == N - 1 or not find(e[1] - 1, i):
                    m_now += 1
            elif find(e[1] - 1, i):
                m_now += 1
        for k in range(N - 2):
            if find(k, i):
                ds += D[k + 1]
        if not (ds == 0 or ds == 2 * len(E)):
            res = m_now / min(ds, 2 * len(E) - ds)
            if res < m:
                m = res
                best = i
    print("cheeger const: " + str(m))
    print("best choice: 1" + format(best, '0' + str(N-2) + 'b')[::-1] + "0")
    return m, best

#### 测试

In [117]:
def test3(N, M):
    E = []
    rnds = np.random.choice(range(N * (N - 1) // 2), M, replace = False)
    count = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            if count in rnds:
                E.append([i, j])
            count += 1
    print("N = " + str(N) +  ", M = " + str(M))
    start = time.time()
    traverse_cheeger(N, E)
    end = time.time()
    print(f"time: {end - start}s")
    print("")

test3(10, 10)
test3(10, 45)
test3(15, 50)
test3(15, 100)
test3(20, 10)
test3(20, 100)
test3(25, 30)
test3(25, 300)

N = 10, M = 10
cheeger const: 0.4
best choice: 1000110100
time: 0.00099945068359375s

N = 10, M = 45
cheeger const: 0.5555555555555556
best choice: 1111100000
time: 0.0019974708557128906s

N = 15, M = 50
cheeger const: 0.4
best choice: 111100110100000
time: 0.08765435218811035s

N = 15, M = 100
cheeger const: 0.5483870967741935
best choice: 101011110000100
time: 0.15214967727661133s

N = 20, M = 10
cheeger const: 0.1111111111111111
best choice: 10001010001000001000
time: 1.157426357269287s

N = 20, M = 100
cheeger const: 0.41414141414141414
best choice: 10110100111011010000
time: 5.332449197769165s

N = 25, M = 30
cheeger const: 0.1724137931034483
best choice: 1101100011000000101101100
time: 72.89438128471375s

N = 25, M = 300
cheeger const: 0.5416666666666666
best choice: 1111111111110000000000000
time: 474.84622955322266s

